## Group 1: merge and biolink format all tsv files
1. with predicate biolink:physically_interacts_with and biolink:gene_associated_with_condition

2. the corresponding config.json file is: config_bigGIM_interacts_with_associated_with

3. all of them have same cols "subject, predicate, object, agent_type, knowledge_level, knowledge_source, object_category, publications, subject_category"

4. Full list of tsv files handled in this group is:


In [151]:
## To do list:
## Add signor and cell_marker_genes.csv into pharmacogenomics KG
    ## Done
## Check overlap with BigGIM KG and the best way to remove redundancy edges
    ## 0 duplicated edges when checking the same combination of subject, object and predicate
## Check which part belongs to AML
    ## 652 edges related to AML

In [152]:
## Load necessary packages
import os
import pandas as pd
import glob
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

## Define the version number
version_number = "01_06_2026"
deployment_date = "2026-01-06"

In [153]:
## Load the Biolink category and predicate dictionary for mapping subject, object, and predicate types
%run ./Biolink_category_and_predication_dictionary.ipynb

Date of last update:  2025-09-04
Order is to always process Node/category map first, since the Edeg/predicate map depends on biolink-complainat node values
-----------------------------------------------------------------------------------------------------------------------------
Dictionary: category_map, Key template: Subject_category or Object_category
------------------------------------------------------------------------------------------
Dictionary: predicate_map, Key template: (Subject_category, Object_category, Predicate)


In [154]:
# print(category_map)

## Load files and convert them into separate node & edge files
* check all imported file structure

In [155]:
## Notice!! Please change the file path of following codes into your own
raw_files_path = '/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/primeKG/dataverse_files/'

## Define the output path for node & edge files after formatting
download_path_node_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/primeKG_parsed_node_{version_number}.tsv'
download_path_edge_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/primeKG_parsed_edge_{version_number}.tsv'

In [156]:
## Check all node files being read
## Read all BigGIM node csv file in group 1

for f in os.listdir(raw_files_path):
    if f.endswith('.csv'):
        print(f)

disease_features.csv
nodes.csv
kg_grouped.csv
kg_grouped_diseases.csv
kg_grouped_diseases_bert_map.csv
kg_giant.csv
kg.csv
drug_features.csv
kg_raw.csv
edges.csv


In [157]:
# -----------------------------------------------------------------------------------------------
# Filename				Description
# -----------------------------------------------------------------------------------------------
# nodes.csv				Contains node level information
# 					Primary key: `node_index`
# -----------------------------------------------------------------------------------------------
# edges.csv				Contains undirected relationships between nodes 
# 					Primary key: (`x_index`, `y_index`)
# -----------------------------------------------------------------------------------------------
# kg.csv					This is the Precision Medicine knowledge graph  
# 					Primary key: (`x_index`, `y_index`)
# -----------------------------------------------------------------------------------------------
# disease_features.csv			Contains textual descriptions of diseases 
# 					Primary key: `node_index`
# -----------------------------------------------------------------------------------------------
# drug_features.csv			Contains textual descriptions of diseases 
# 					Primary key: `node_index`
# -----------------------------------------------------------------------------------------------
# kg_raw.csv 				Intermediate PrimeKG made by joining nodes and edges
# -----------------------------------------------------------------------------------------------
# kg_giant.csv				Intermediate PrimeKG made by taking LCC of kg_raw.csv 
# -----------------------------------------------------------------------------------------------
# kg_grouped.csv				Intermediate PrimeKG made by grouping diseases  
# -----------------------------------------------------------------------------------------------
# kg_grouped_diseases.csv			List of all diseases and their assigned group name  
# -----------------------------------------------------------------------------------------------
# kg_grouped_diseases_bert_map.csv	Manual grouping created for diseases using BERT model
# ---------------------------------------------------------------------------------------------

In [158]:
## Read each individual csv files
nodes_df = pd.read_csv(raw_files_path + 'nodes.csv')

nodes_df.head(10)

,node_index,node_id,node_type,node_name,node_source
0,0,9796,gene/protein,PHYHIP,NCBI
1,1,7918,gene/protein,GPANK1,NCBI
2,2,8233,gene/protein,ZRSR2,NCBI
3,3,4899,gene/protein,NRF1,NCBI
4,4,5297,gene/protein,PI4KA,NCBI
5,5,6564,gene/protein,SLC15A1,NCBI
6,6,8668,gene/protein,EIF3I,NCBI
7,7,10826,gene/protein,FAXDC2,NCBI
8,8,4489,gene/protein,MT1A,NCBI
9,9,6272,gene/protein,SORT1,NCBI


In [159]:
## check unique node_type and their node_source
unique_node_type_values = nodes_df['node_type'].unique()
print("All possible node_type are here: " ,unique_node_type_values)

unique_node_source_values = nodes_df['node_source'].unique()
print("All possible node_source are here: " ,unique_node_source_values)

All possible node_type are here:  ['gene/protein' 'drug' 'effect/phenotype' 'disease' 'biological_process'
 'molecular_function' 'cellular_component' 'exposure' 'pathway' 'anatomy']
All possible node_source are here:  ['NCBI' 'DrugBank' 'HPO' 'MONDO_grouped' 'MONDO' 'GO' 'CTD' 'REACTOME'
 'UBERON']


### Implementation notes:
* Essential list of nodes to add: gene/protein, drug, disease, effect/phenotype
* Good to have list: pathway, exposure (more on the enviromental side), anatomy (only relevant if we need spatial information, e.g. cancer on specific organ)
* Maybe not for now?: biological_process, molecular_function, cellular_component
*
* following example inputs here for node normalizer service:
* https://github.com/TranslatorSRI/NodeNormalization/blob/master/documentation/NodeNormalization.ipynb
* 
* for MONDO codes: need to change format to MONDO:XXXXXXX, most cases those numbers are not in correct 7 digits so need to padding zeros
* Discussion: how to deal with MONDO_grouped?
* see question 1 below
* for NCBI code: add NCBIGene: prefix
* for HPO code: also need to change format to HP:XXXXXXX, 7 digits
* for DrugBank: need to check if Translator accepts DB: codes
* for GO: also need to change format to HP:XXXXXXX, 7 digits
* for CTD: change format to CID:, add prefix
* for REACTOME:
* for UBERON: also need to change format to HP:XXXXXXX, 7 digits

In [160]:
sanity_check_df = nodes_df[nodes_df['node_type'] == 'effect/phenotype']

sanity_check_df.head(15)

,node_index,node_id,node_type,node_name,node_source
22117,22117,1507,effect/phenotype,Growth abnormality,HPO
22118,22118,107,effect/phenotype,Renal cyst,HPO
22119,22119,1,effect/phenotype,All,HPO
22120,22120,5,effect/phenotype,Mode of inheritance,HPO
22121,22121,10460,effect/phenotype,Abnormality of the female genitalia,HPO
22122,22122,812,effect/phenotype,Abnormal internal genitalia,HPO
22123,22123,14,effect/phenotype,Abnormality of the bladder,HPO
22124,22124,2719,effect/phenotype,Recurrent infections,HPO
22125,22125,11277,effect/phenotype,Abnormality of the urinary system physiology,HPO
22126,22126,8684,effect/phenotype,Aplasia/hypoplasia of the uterus,HPO


In [161]:
## Sanity check on the current format of the file

# Randomly select one row for each unique node_type
sampled_df = nodes_df.groupby('node_type').sample(n=1, random_state=151)  # Set random_state for reproducibility

print(sampled_df)

        node_index        node_id           node_type  \
65751        65751           3476             anatomy   
114674      114674        1902367  biological_process   
124851      124851          70913  cellular_component   
99417        99417          23137             disease   
19830        19830        DB06980                drug   
87918        87918           9598    effect/phenotype   
61700        61700        C091375            exposure   
34206        34206         348793        gene/protein   
115725      115725          36200  molecular_function   
128395      128395  R-HSA-1368082             pathway   

                                                node_name node_source  
65751              respiratory system venous blood vessel      UBERON  
114674  negative regulation of Notch signaling pathway...          GO  
124851                                 Ddb1-Wdr21 complex          GO  
99417                        feigenbaum Bergeron syndrome       MONDO  
19830       

### Following are questions for data ingestion of the PrimeKG into Pharmocogenomics KG
* **Question 1:** How to handle those grouped Mondo codes?
* Remove them for now!

In [162]:
nodes_MONDO_grouped = nodes_df[nodes_df['node_source'] == 'MONDO_grouped']

nodes_MONDO_grouped.head(10)

,node_index,node_id,node_type,node_name,node_source
27158,27158,13924_12592_14672_13460_12591_12536_30861_8146...,disease,osteogenesis imperfecta,MONDO_grouped
27159,27159,11160_13119_13978_12060_12327_12670_13210_1106...,disease,autosomal recessive nonsyndromic deafness,MONDO_grouped
27160,27160,8099_12497_12498,disease,congenital stationary night blindness autosoma...,MONDO_grouped
27161,27161,14854_14293_14470_12380_11832_14603_14853_1176...,disease,autosomal dominant nonsyndromic deafness,MONDO_grouped
27162,27162,33202_32776_30905_33670_33200_32740_32732_3320...,disease,"deafness, autosomal recessive",MONDO_grouped
27163,27163,11396_7422,disease,keratoderma hereditarium mutilans,MONDO_grouped
27164,27164,14828_14829_9454_13553_133,disease,immunodeficiency-centromeric instability-facia...,MONDO_grouped
27167,27167,9260_9261_9262_18149,disease,GM1 gangliosidosis,MONDO_grouped
27170,27170,14083_13288_12987_13287_13289_20729_14840_1329...,disease,agammaglobulinemia,MONDO_grouped
27173,27173,14986_14987_10351_10953_13565_9213_12565_13248...,disease,Fanconi anemia complementation group,MONDO_grouped


* **Questions 2:** How do or should we want to integrate those additional feature information of drug & disease into our KG?
* Not using those features for now!!!
* Sample information can be obtained from disease features
    - mondo_id	mondo_name	group_id_bert	group_name_bert	mondo_definition	umls_description	orphanet_definition	**orphanet_prevalence**	orphanet_epidemiology	orphanet_clinical_description	**orphanet_management_and_treatment**	mayo_symptoms	mayo_causes	mayo_risk_factors	mayo_complications	mayo_prevention	mayo_see_doc
    - 12345	acral peeling skin syndrome			Acral peeling skin syndrome (PSS) is a form of PSS characterized by superficial peeling of the skin predominantly affecting the dorsa of the hands and feet.		A rare peeling skin syndrome characterized by superficial peeling of the skin predominantly affecting the dorsa of the hands and feet.	**<1/1000000**	Acral PSS is rare, with approximately 40 cases described in the literature to date.	The disease manifests shortly after birth or in early childhood with superficial peeling on the palmar, plantar and dorsal surfaces of the hands and feet, that leaves residual painless erythema. Manual skin removal is also possible. Seasonal variations are generally observed. Heat, humidity, exposure to water and friction or minor trauma can induce exfoliation. The lesions are not painful and heal without scarring.	**There is no effective treatment.** Emollients are often used to reduce skin peeling. Patients must avoid immersion in water and are recommended to use absorbing powders or aluminum antiperspirants.						
* Sample information can be obtained from drug features
    - description	**half_life**	indication	**mechanism_of_action**	protein_binding	pharmacodynamics	state	atc_1	atc_2	atc_3	atc_4	category	group	pathway	molecular_weight	tpsa	**clogp**
    - Budesonide is a glucocorticoid that is a mix of the 22R and 22S epimer used to treat inflammatory conditions of the lungs and intestines such as asthma, COPD, Crohn's disease, and ulcerative colitis.	**Budesonide has a plasma elimination half life of 2-3.6h. The terminal elimination half life in asthmatic children 4-6 years old is 2.3h.**	Budesonide extended release capsules are indicated for the treatment and maintenance of mild to moderate Crohn’s disease. Various inhaled budesonide products are indicated for prophylactic therapy in asthma and reducing exacerbations of COPD. A budesonide nasal spray is available over the counter for symptoms of hay fever and upper respiratory allergies. Extended release capsules are indicated to induce remission of mild to moderate ulcerative colitis and a rectal foam is used for mild to moderate distal ulcerative colitis.	**The short term effects of corticosteroids are decreased vasodilation and permeability of capillaries, as well as decreased leukocyte migration to sites of inflammation.** Corticosteroids binding to the glucocorticoid receptor mediates changes in gene expression that lead to multiple downstream effects over hours to days.	Corticosteroids are generally bound to corticosteroid binding globulin and serum albumin in plasma. Budesonide is 85-90% protein bound in plasma.	Budesonide is a glucocorticoid used to treat respiratory and digestive conditions by reducing inflammation. It has a wide therapeutic index, as dosing varies highly from patient to patient. Patients should be counselled regarding the risk of hypercorticism and adrenal axis suppression.	Budesonide is a solid.	Budesonide is anatomically related to dermatologicals and respiratory system and respiratory system and alimentary tract and metabolism and respiratory system.	Budesonide is in the therapeutic group of corticosteroids, dermatological preparations and nasal preparations and drugs for obstructive airway diseases and antidiarrheals, intestinal antiinflammatory/antiinfective agents and drugs for obstructive airway diseases.	Budesonide is pharmacologically related to corticosteroids, plain and decongestants and other nasal preparations for topical use and adrenergics, inhalants and intestinal antiinflammatory agents and other drugs for obstructive airway diseases, inhalants.	The chemical and functional group of  is corticosteroids, potent (group iii) and corticosteroids and adrenergics in combination with corticosteroids or other drugs, excl. anticholinergics and corticosteroids acting locally and glucocorticoids.	Budesonide is part of Adrenal Cortex Hormones ; Adrenals ; Agents to Treat Airway Disease ; Alimentary Tract and Metabolism ; Anti-Asthmatic Agents ; Anti-Inflammatory Agents ; Antidiarrheals, Intestinal Antiinflammatory/antiinfective Agents ; Autonomic Agents ; Bronchodilator Agents ; BSEP/ABCB11 Substrates ; Corticosteroid Hormone Receptor Agonists ; Corticosteroids ; Corticosteroids Acting Locally ; Corticosteroids for Systemic Use ; Corticosteroids, Dermatological Preparations ; Corticosteroids, Potent (Group III) ; Cytochrome P-450 CYP2A6 Inducers ; Cytochrome P-450 CYP2B6 Inducers ; Cytochrome P-450 CYP2B6 Inducers (strength unknown) ; Cytochrome P-450 CYP2C19 Inducers ; Cytochrome P-450 CYP2C19 Inducers (strength unknown) ; Cytochrome P-450 CYP2C8 Inducers ; Cytochrome P-450 CYP2C8 Inducers (strength unknown) ; Cytochrome P-450 CYP2C9 Inducers ; Cytochrome P-450 CYP2C9 Inducers (strength unknown) ; Cytochrome P-450 CYP3A Inducers ; Cytochrome P-450 CYP3A Substrates ; Cytochrome P-450 CYP3A4 Inducers ; Cytochrome P-450 CYP3A4 Inducers (strength unknown) ; Cytochrome P-450 CYP3A4 Substrates ; Cytochrome P-450 CYP3A5 Inducers ; Cytochrome P-450 CYP3A5 Inducers (moderate) ; Cytochrome P-450 Enzyme Inducers ; Cytochrome P-450 Substrates ; Dermatologicals ; Drugs for Obstructive Airway Diseases ; Drugs that are Mainly Renally Excreted ; Fused-Ring Compounds ; Hormones ; Hormones, Hormone Substitutes, and Hormone Antagonists ; Immunosuppressive Agents ; Intestinal Antiinflammatory Agents ; Nasal Preparations ; OAT3/SLC22A8 Substrates ; P-glycoprotein substrates ; Peripheral Nervous System Agents ; Pregnanes ; Pregnenediones ; Pregnenes ; Respiratory System Agents ; Steroids.	Budesonide is approved.		The molecular weight is 430.54.	Budesonide has a topological polar surface area of 93.06.	**The log p value of  is 2.9.**
    - clogp: a predicted value for the partition coefficient (LogP) of a molecule

In [163]:
# ## Read each individual csv files
# disease_features_df = pd.read_csv(raw_files_path + 'disease_features.csv')

# disease_features_df.head(5)

In [164]:
# ## Read each individual csv files
# drug_features_df = pd.read_csv(raw_files_path + 'drug_features.csv')

# drug_features_df.head(5)

In [165]:
## Select only the 4 must have node types
must_have_list = ['gene/protein', 'drug', 'disease', 'effect/phenotype']
nodes_select_df = nodes_df[nodes_df['node_type'].isin(must_have_list)]

## count rows
unique_node_count = nodes_select_df['node_index'].nunique()
print(unique_node_count)

68019


In [166]:
sanity_check_df = nodes_select_df[nodes_select_df['node_id'] == '5156']

sanity_check_df.head()

,node_index,node_id,node_type,node_name,node_source
3772,3772,5156,gene/protein,PDGFRA,NCBI
36722,36722,5156,disease,encephalomyelitis,MONDO
86127,86127,5156,effect/phenotype,Hypoplastic left atrium,HPO


In [167]:
## let's exclude those 'MONDO_grouped' rows for now
nodes_select_no_MONDO_grouped_df = nodes_select_df[nodes_select_df['node_source'] != 'MONDO_grouped']

## count rows
unique_node_count = nodes_select_no_MONDO_grouped_df['node_index'].nunique()
print(unique_node_count)

66752


In [168]:
## get the list of unique values in column node_id of nodes_select_no_MONDO_grouped_df
## since node_id neededs to combine with node_source as prefix to obtain a unique identifier
## see above example of same "node_id" in multiple vocabularies
unique_node_ids = nodes_select_no_MONDO_grouped_df['node_index'].unique().tolist()

## Read each individual csv files
edges_df = pd.read_csv(raw_files_path + 'kg.csv')

## count rows
print(len(edges_df))

## Filter to only include edges that are in the selected node_id list
edges_filtered_df = edges_df[edges_df['x_index'].isin(unique_node_ids)]
edges_filtered2_df = edges_filtered_df[edges_filtered_df['y_index'].isin(unique_node_ids)]

## count rows
print(len(edges_filtered2_df))

edges_filtered2_df.head(5)

/var/folders/ml/cjwyk2ps62361rr96_3b_n080000gp/T/ipykernel_17326/2807197227.py:7: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(raw_files_path + 'kg.csv')


8100498
4009644


,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source
0,protein_protein,ppi,0,9796,gene/protein,PHYHIP,NCBI,8889,56992,gene/protein,KIF15,NCBI
1,protein_protein,ppi,1,7918,gene/protein,GPANK1,NCBI,2798,9240,gene/protein,PNMA1,NCBI
2,protein_protein,ppi,2,8233,gene/protein,ZRSR2,NCBI,5646,23548,gene/protein,TTC33,NCBI
3,protein_protein,ppi,3,4899,gene/protein,NRF1,NCBI,11592,11253,gene/protein,MAN1B1,NCBI
4,protein_protein,ppi,4,5297,gene/protein,PI4KA,NCBI,2122,8601,gene/protein,RGS20,NCBI


In [169]:
## check unique relation
unique_relation_values = edges_filtered2_df['relation'].value_counts()
print("All possible relation are here: " , unique_relation_values)

All possible relation are here:  relation
drug_drug                     2672628
protein_protein                642150
disease_phenotype_positive     222000
drug_effect                    129568
disease_protein                127408
drug_protein                    51306
contraindication                49924
disease_disease                 49452
phenotype_phenotype             37472
indication                      15318
phenotype_protein                6660
off-label use                    4372
disease_phenotype_negative       1386
Name: count, dtype: int64


In [170]:
## check unique relation
unique_relation_values = edges_filtered2_df['display_relation'].value_counts()
print("All possible display relation are here: " , unique_relation_values)

All possible display relation are here:  display_relation
synergistic interaction    2672628
ppi                         642150
phenotype present           222000
associated with             134068
side effect                 129568
parent-child                 86924
contraindication             49924
target                       32760
indication                   15318
enzyme                       10634
transporter                   6184
off-label use                 4372
carrier                       1728
phenotype absent              1386
Name: count, dtype: int64


In [171]:
## check unique relation
unique_type_values = set(list(edges_filtered2_df['x_type'].unique()) + list(edges_filtered2_df['y_type'].unique()))
print("All possible node types are here: " , unique_type_values)

All possible node types are here:  {'gene/protein', 'drug', 'disease', 'effect/phenotype'}


In [172]:
## check unique relation
unique_type_values = set(list(edges_filtered2_df['x_source'].unique()) + list(edges_filtered2_df['y_source'].unique()))
print("All possible node sources are here: " , unique_type_values)

All possible node sources are here:  {'HPO', 'MONDO', 'NCBI', 'DrugBank'}


In [173]:
## get all unqiue combinations of subject, object, and relation
## check all unique combinations between subject & object
from itertools import combinations

# Get all combinations of 3 column names
col_combos = ['x_type', 'y_type', 'relation']

unique_combinations = edges_filtered2_df[col_combos].drop_duplicates()

print(len(unique_combinations))
print(unique_combinations)

22
                   x_type            y_type                    relation
0            gene/protein      gene/protein             protein_protein
321075               drug      gene/protein                drug_protein
346728               drug           disease            contraindication
346730               drug           disease                  indication
347016               drug           disease               off-label use
389359               drug              drug                   drug_drug
3061987      gene/protein  effect/phenotype           phenotype_protein
3065317  effect/phenotype  effect/phenotype         phenotype_phenotype
3084066           disease  effect/phenotype  disease_phenotype_negative
3085282           disease  effect/phenotype  disease_phenotype_positive
3234966  effect/phenotype           disease  disease_phenotype_positive
3236466      gene/protein           disease             disease_protein
3315993           disease           disease             disea

In [174]:
## get all unqiue combinations of subject, object, and relation
## check all unique combinations between subject & object
from itertools import combinations

# Get all combinations of 3 column names
col_combos = ['x_type', 'y_type', 'display_relation']

unique_combinations = edges_filtered2_df[col_combos].drop_duplicates()

print(len(unique_combinations))
print(unique_combinations)

28
                   x_type            y_type         display_relation
0            gene/protein      gene/protein                      ppi
321075               drug      gene/protein                  carrier
321939               drug      gene/protein                   enzyme
327256               drug      gene/protein                   target
343636               drug      gene/protein              transporter
346728               drug           disease         contraindication
346730               drug           disease               indication
347016               drug           disease            off-label use
389359               drug              drug  synergistic interaction
3061987      gene/protein  effect/phenotype          associated with
3065317  effect/phenotype  effect/phenotype             parent-child
3084066           disease  effect/phenotype         phenotype absent
3085282           disease  effect/phenotype        phenotype present
3234966  effect/phenotype      

In [175]:
## uncertain mapping
##  drug      gene/protein                   enzyme
## gene/protein              drug                   enzyme

In [176]:
## Sanity check anatomy_anatomy relations
## should return empty df
sanity_check_df = edges_filtered2_df[edges_filtered2_df['relation'] == 'anatomy_anatomy']
sanity_check_df.head(10)

,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source


### Investigate those reverse order pairs of subject & objects
* seem like in raw PrimeKG file, they kept a reverse copy of all those edges
* Strategy is to throw away those pairs that "contradict" to what defined in the biolink associations & predicates

In [177]:
sanity_check_df = edges_filtered2_df[ (edges_filtered2_df['display_relation'] == 'transporter') & (edges_filtered2_df['x_type'] == 'drug')]

print(len(sanity_check_df))
# sanity_check_df.head(5)

3092


In [178]:
sanity_check_df = edges_filtered2_df[ (edges_filtered2_df['display_relation'] == 'transporter') & (edges_filtered2_df['x_type'] == 'gene/protein')]

print(len(sanity_check_df))
# sanity_check_df.head(5)

3092


In [179]:
sanity_check_df = edges_filtered2_df[ (edges_filtered2_df['display_relation'] == 'target') & (edges_filtered2_df['x_type'] == 'drug')]

print(len(sanity_check_df))
# sanity_check_df.head(5)

16380


In [180]:
sanity_check_df = edges_filtered2_df[ (edges_filtered2_df['display_relation'] == 'target') & (edges_filtered2_df['x_type'] == 'gene/protein')]

print(len(sanity_check_df))
sanity_check_df.head(5)

16380


,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source
5714056,drug_protein,target,13174,3067,gene/protein,HDC,NCBI,15906,DB00114,drug,Pyridoxal phosphate,DrugBank
5714057,drug_protein,target,13174,3067,gene/protein,HDC,NCBI,15907,DB00117,drug,Histidine,DrugBank
5714058,drug_protein,target,13982,27165,gene/protein,GLS2,NCBI,15908,DB00142,drug,Glutamic acid,DrugBank
5714059,drug_protein,target,476,2162,gene/protein,F13A1,NCBI,15909,DB02340,drug,N-Acetyl-Serine,DrugBank
5714060,drug_protein,target,476,2162,gene/protein,F13A1,NCBI,15910,DB11300,drug,Thrombin,DrugBank


In [181]:
sanity_check_df = edges_filtered2_df[ (edges_filtered2_df['display_relation'] == 'off-label use') & (edges_filtered2_df['x_type'] == 'drug')]

print(len(sanity_check_df))
# sanity_check_df.head(5)

2186


In [182]:
sanity_check_df = edges_filtered2_df[ (edges_filtered2_df['display_relation'] == 'off-label use') & (edges_filtered2_df['x_type'] == 'disease')]

print(len(sanity_check_df))
sanity_check_df.head(5)

2186


,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source
5733816,off-label use,off-label use,33577,5044,disease,hypertensive disorder,MONDO,14257,DB00903,drug,Etacrynic acid,DrugBank
5734096,off-label use,off-label use,33577,5044,disease,hypertensive disorder,MONDO,14668,DB00887,drug,Bumetanide,DrugBank
5734351,off-label use,off-label use,38121,5391,disease,restless legs syndrome,MONDO,14321,DB01235,drug,Levodopa,DrugBank
5734353,off-label use,off-label use,38121,5391,disease,restless legs syndrome,MONDO,19207,DB00190,drug,Carbidopa,DrugBank
5734665,off-label use,off-label use,28396,7803,disease,multiple system atrophy,MONDO,15003,DB01380,drug,Cortisone acetate,DrugBank


### Now handles the different type of nodes differently
* following example inputs here for node normalizer service: https://github.com/TranslatorSRI/NodeNormalization/blob/master/documentation/NodeNormalization.ipynb
* Drug: DRUGBANK:DB09145
* Disease: MONDO:0004976
* Gene / Protein: NCBIGene:9496
* Effect / Phenotype: HP:0007354
* Need to find a way to add zeros padding for MONDO and HP codes

In [183]:
## Start with check MONDO or HPO in either x_source or y_source
## this is just for test run, when the code is done, directly run on full df
mondo_or_hpo = ['MONDO', 'HPO']
edges_mondo_hpo_df = edges_filtered2_df[ (edges_filtered2_df['x_source'].isin(mondo_or_hpo)) | (edges_filtered2_df['y_source'].isin(mondo_or_hpo))]

# Randomly select one row for each unique combination
sampled_df = edges_mondo_hpo_df.groupby(['x_source', 'y_source']).sample(n=1, random_state=151)  # Set random_state for reproducibility
# sampled_df

In [184]:
## work directly on the full dataframe
## first loop x_id 
## to add prefix MONDO or HP for rows with sources "MONDO" or "HPO" and padding to 7 digits as required by example

## deep copy a df
edges_deep_copy_df = edges_filtered2_df.copy(deep=True)

## Prefix mapping for selected sources
prefix_map = {
    'HPO': 'HP:',
    'MONDO': 'MONDO:',
    'DrugBank': 'DRUGBANK:', 
    'NCBI': 'NCBIGene:'
}

## Initialize new column with default values (original id as string)
edges_deep_copy_df['subject'] = edges_deep_copy_df['x_id'].astype(str)

## Mask for HPO and MONDO → prefix + zero-padded id
mask_hpo_mondo = edges_deep_copy_df['x_source'].isin(['HPO', 'MONDO'])
edges_deep_copy_df.loc[mask_hpo_mondo, 'subject'] = (
    edges_deep_copy_df.loc[mask_hpo_mondo, 'x_source'].map(prefix_map) +
    edges_deep_copy_df.loc[mask_hpo_mondo, 'x_id'].astype(str).str.zfill(7)
)

## Mask for DrugBank → prefix only, no padding
mask_drugbank_ncbi = edges_deep_copy_df['x_source'].isin(['DrugBank', 'NCBI'])
edges_deep_copy_df.loc[mask_drugbank_ncbi, 'subject'] = (
    edges_deep_copy_df.loc[mask_drugbank_ncbi, 'x_source'].map(prefix_map) +
    edges_deep_copy_df.loc[mask_drugbank_ncbi, 'x_id'].astype(str)
)

sampled_df = edges_deep_copy_df.groupby(['x_source']).sample(n=1, random_state=151)  # Set random_state for reproducibility
sampled_df

,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source,subject
1362595,drug_drug,synergistic interaction,21014,DB13648,drug,Alcuronium,DrugBank,14614,DB06738,drug,Ketobemidone,DrugBank,DRUGBANK:DB13648
5891379,disease_phenotype_positive,phenotype present,88949,11800,effect/phenotype,Midface retrusion,HPO,31464,9841,disease,PEHO syndrome,MONDO,HP:0011800
3153485,disease_phenotype_positive,phenotype present,31611,14892,disease,micrognathia-recurrent infections-behavioral a...,MONDO,22715,750,effect/phenotype,Delayed speech and language development,HPO,MONDO:0014892
116373,protein_protein,ppi,2071,27074,gene/protein,LAMP3,NCBI,10788,347733,gene/protein,TUBB2B,NCBI,NCBIGene:27074


In [185]:
## similarly change for y_id
## Initialize new column with default values (original id as string)
edges_deep_copy_df['object'] = edges_deep_copy_df['y_id'].astype(str)

## Mask for HPO and MONDO → prefix + zero-padded id
mask_hpo_mondo = edges_deep_copy_df['y_source'].isin(['HPO', 'MONDO'])
edges_deep_copy_df.loc[mask_hpo_mondo, 'object'] = (
    edges_deep_copy_df.loc[mask_hpo_mondo, 'y_source'].map(prefix_map) +
    edges_deep_copy_df.loc[mask_hpo_mondo, 'y_id'].astype(str).str.zfill(7)
)

## Mask for DrugBank → prefix only, no padding
mask_drugbank_ncbi = edges_deep_copy_df['y_source'].isin(['DrugBank', 'NCBI'])
edges_deep_copy_df.loc[mask_drugbank_ncbi, 'object'] = (
    edges_deep_copy_df.loc[mask_drugbank_ncbi, 'y_source'].map(prefix_map) +
    edges_deep_copy_df.loc[mask_drugbank_ncbi, 'y_id'].astype(str)
)

sampled_df = edges_deep_copy_df.groupby(['y_source']).sample(n=1, random_state=151)  # Set random_state for reproducibility
sampled_df

,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source,subject,object
1423055,drug_drug,synergistic interaction,15103,DB00496,drug,Darifenacin,DrugBank,15441,DB00402,drug,Eszopiclone,DrugBank,DRUGBANK:DB00496,DRUGBANK:DB00402
3200343,disease_phenotype_positive,phenotype present,28319,8900,disease,camptodactyly with fibrous tissue hyperplasia ...,MONDO,22650,100490,effect/phenotype,Camptodactyly of finger,HPO,MONDO:0008900,HP:0100490
3249032,disease_protein,associated with,1749,7150,gene/protein,TOP1,NCBI,84244,16267,disease,undifferentiated carcinoma of the corpus uteri,MONDO,NCBIGene:7150,MONDO:0016267
116373,protein_protein,ppi,2071,27074,gene/protein,LAMP3,NCBI,10788,347733,gene/protein,TUBB2B,NCBI,NCBIGene:27074,NCBIGene:347733


* **Questions 3:** for effect/phenotype nodes which biolink node type to map to
* following are detail on candidates
* DiseaseOrPhenotypicFeature
  - Either one of a disease or an individual phenotypic feature. Some knowledge resources such as Monarch treat these as distinct, others such as MESH conflate. Please see definitions of phenotypic feature and disease in this model for their independent descriptions. This class is helpful to enforce domains and ranges that may involve either a disease or a phenotypic feature.
* DiseaseOrPhenotypicFeatureExposure
  - A disease or phenotypic feature state, when viewed as an exposure, represents an precondition, leading to or influencing an outcome, e.g. HIV predisposing an individual to infections; a relative deficiency of skin pigmentation predisposing an individual to skin cancer.
* DiseaseOrPhenotypicFeatureOutcome
  - Physiological outcomes resulting from an exposure event which is the manifestation of a disease or other characteristic phenotype.

### Load the Biolink category and predicate dictionary for mapping subject, object, and predicate types

In [186]:
## Load the Biolink category and predicate dictionary for mapping subject, object, and predicate types
%run ./Biolink_category_and_predication_dictionary.ipynb

Date of last update:  2025-09-04
Order is to always process Node/category map first, since the Edeg/predicate map depends on biolink-complainat node values
-----------------------------------------------------------------------------------------------------------------------------
Dictionary: category_map, Key template: Subject_category or Object_category
------------------------------------------------------------------------------------------
Dictionary: predicate_map, Key template: (Subject_category, Object_category, Predicate)


In [187]:
## deep copy a df
edges_deep_copy_df2 = edges_deep_copy_df.copy(deep=True)

## assign biolink node category using dictionary directly
edges_deep_copy_df2['subject_category'] = (
    edges_deep_copy_df2['x_type'].map(category_map)
)

edges_deep_copy_df2['object_category'] = (
    edges_deep_copy_df2['y_type'].map(category_map)
)

sampled_df = edges_deep_copy_df2.groupby(['x_type']).sample(n=1, random_state=151)  # Set random_state for reproducibility
sampled_df

,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source,subject,object,subject_category,object_category
3157330,disease_phenotype_positive,phenotype present,31809,32864,disease,intellectual developmental disorder with speec...,MONDO,22759,6,effect/phenotype,Autosomal dominant inheritance,HPO,MONDO:0032864,HP:0000006,biolink:Disease,biolink:PhenotypicFeature
1162436,drug_drug,synergistic interaction,20349,DB09543,drug,Methyl salicylate,DrugBank,14432,DB09383,drug,Meprednisone,DrugBank,DRUGBANK:DB09543,DRUGBANK:DB09383,biolink:Drug,biolink:Drug
6112214,drug_effect,side effect,22939,83,effect/phenotype,Renal insufficiency,HPO,17730,DB00710,drug,Ibandronate,DrugBank,HP:0000083,DRUGBANK:DB00710,biolink:PhenotypicFeature,biolink:Drug
5473379,protein_protein,ppi,637,5725,gene/protein,PTBP1,NCBI,866,5595,gene/protein,MAPK3,NCBI,NCBIGene:5725,NCBIGene:5595,biolink:Gene,biolink:Gene


In [188]:
## Sanity check anatomy_anatomy relations
## should return empty df after mapping to dictionary
sanity_check_df = edges_deep_copy_df2[edges_deep_copy_df2['display_relation'] == 'side effect']
print(len(sanity_check_df))
sanity_check_df.head(5)

129568


,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source,subject,object,subject_category,object_category
3348335,drug_effect,side effect,16322,DB00583,drug,Levocarnitine,DrugBank,23158,2027,effect/phenotype,Abdominal pain,HPO,DRUGBANK:DB00583,HP:0002027,biolink:Drug,biolink:PhenotypicFeature
3348336,drug_effect,side effect,16322,DB00583,drug,Levocarnitine,DrugBank,85849,4396,effect/phenotype,Poor appetite,HPO,DRUGBANK:DB00583,HP:0004396,biolink:Drug,biolink:PhenotypicFeature
3348337,drug_effect,side effect,16322,DB00583,drug,Levocarnitine,DrugBank,22447,739,effect/phenotype,Anxiety,HPO,DRUGBANK:DB00583,HP:0000739,biolink:Drug,biolink:PhenotypicFeature
3348338,drug_effect,side effect,16322,DB00583,drug,Levocarnitine,DrugBank,22831,11675,effect/phenotype,Arrhythmia,HPO,DRUGBANK:DB00583,HP:0011675,biolink:Drug,biolink:PhenotypicFeature
3348339,drug_effect,side effect,16322,DB00583,drug,Levocarnitine,DrugBank,23469,3418,effect/phenotype,Back pain,HPO,DRUGBANK:DB00583,HP:0003418,biolink:Drug,biolink:PhenotypicFeature


In [189]:
## Sanity check anatomy_anatomy relations
## should return empty df after mapping to dictionary
## enzyme: for drug metabolism
sanity_check_df = edges_deep_copy_df2[edges_deep_copy_df2['display_relation'] == 'enzyme']
sanity_check_df.head(5)

,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source,subject,object,subject_category,object_category
321939,drug_protein,enzyme,14584,DB00130,drug,L-Glutamine,DrugBank,13982,27165,gene/protein,GLS2,NCBI,DRUGBANK:DB00130,NCBIGene:27165,biolink:Drug,biolink:Gene
321940,drug_protein,enzyme,14585,DB11118,drug,Ammonia,DrugBank,13982,27165,gene/protein,GLS2,NCBI,DRUGBANK:DB11118,NCBIGene:27165,biolink:Drug,biolink:Gene
321941,drug_protein,enzyme,14584,DB00130,drug,L-Glutamine,DrugBank,476,2162,gene/protein,F13A1,NCBI,DRUGBANK:DB00130,NCBIGene:2162,biolink:Drug,biolink:Gene
321942,drug_protein,enzyme,14275,DB00997,drug,Doxorubicin,DrugBank,373,4843,gene/protein,NOS2,NCBI,DRUGBANK:DB00997,NCBIGene:4843,biolink:Drug,biolink:Gene
321943,drug_protein,enzyme,14423,DB09237,drug,Levamlodipine,DrugBank,373,4843,gene/protein,NOS2,NCBI,DRUGBANK:DB09237,NCBIGene:4843,biolink:Drug,biolink:Gene


* **Question 4:** general mapping for predicate
* protein-protein interacts, use physical_interacts_with
* carrier, use 'biolink:can_be_carried_out_by'?
* enzyme?
* indication?
* synergistic interaction?
* parent-child, between two phenotypes nodes, should I use biolink:broad_match?
* phenotype absent, opposite of has_phenotype? or just ignore?

In [190]:
## Group by predicate
grouped = edges_deep_copy_df2.groupby('display_relation')

## For each predicate, output unique (subject_category, object_category) pairs
for predicate, group in grouped:
    print(f"\nPredicate: {predicate}")
    pairs = group[['subject_category', 'object_category']].drop_duplicates()
    for _, row in pairs.iterrows():
        print(f"  ({row['subject_category']}, {row['object_category']})")


Predicate: associated with
  (biolink:Gene, biolink:PhenotypicFeature)
  (biolink:Gene, biolink:Disease)
  (biolink:PhenotypicFeature, biolink:Gene)
  (biolink:Disease, biolink:Gene)

Predicate: carrier
  (biolink:Drug, biolink:Gene)
  (biolink:Gene, biolink:Drug)

Predicate: contraindication
  (biolink:Drug, biolink:Disease)
  (biolink:Disease, biolink:Drug)

Predicate: enzyme
  (biolink:Drug, biolink:Gene)
  (biolink:Gene, biolink:Drug)

Predicate: indication
  (biolink:Drug, biolink:Disease)
  (biolink:Disease, biolink:Drug)

Predicate: off-label use
  (biolink:Drug, biolink:Disease)
  (biolink:Disease, biolink:Drug)

Predicate: parent-child
  (biolink:PhenotypicFeature, biolink:PhenotypicFeature)
  (biolink:Disease, biolink:Disease)

Predicate: phenotype absent
  (biolink:Disease, biolink:PhenotypicFeature)
  (biolink:PhenotypicFeature, biolink:Disease)

Predicate: phenotype present
  (biolink:Disease, biolink:PhenotypicFeature)
  (biolink:PhenotypicFeature, biolink:Disease)

Pred

In [192]:
## start convert to biolink compliant predicate types based on display_relation column
## check supplementary table of primeKG, descriptions on relations
## translator discussion point - data ingestion group meeting
## separate out off label drug usage, clinical trial, and FDA approved usage
## for 'synergistic interaction', combine with knowledge from LLM clinical trial attempts and find drug combination and which disease 
## they targeted
## currently exclude associated with
## not need to include 'phenotype absent'

## match only combinations allowed in the pair dictionary
## Apply the mapping and return a Series with 3 columns
edges_deep_copy_df2[['predicate', 'subject_direction_qualifier', 'object_direction_qualifier', 'object_aspect_qualifier']] = edges_deep_copy_df2.apply(
    lambda row: pd.Series(
        predicate_map.get(
            (row['subject_category'], row['object_category'], row['display_relation']),
            [None, None, None]  # Default if not found
        )
    ),
    axis=1
)

## add individual data source (original instead of PrimeKG) for each edge in the PrimeKG
* The mapping can be found in the build_kg script: https://github.com/mims-harvard/PrimeKG/blob/main/knowledge_graph/build_graph.ipynb
* Then we create our own dictionary to map the individual data souce to the ingested KG
* use a unique combon of following columns:
* x_source, y_source, relation

In [193]:
## temp: use the whole PrimeKG as knowledge source for now
# data_source_map_dict = {
#     ('NCBI', 'NCBI', 'protein_protein'): 'NCBI',
#     ('NCBI', 'MONDO', 'disease_protein'): 'DisGenNet',
#     ('NCBI', 'HPO', 'phenotype_protein'): 'DisGenNet',
#     ('NCBI', 'GO', 'molfunc_protein'): 'Gene2GO',
#     ('NCBI', 'GO', 'cellcomp_protein'): 'Gene2GO',
#     ('NCBI', 'GO', 'bioprocess_protein'): 'Gene2GO',
#     ('NCBI', 'UBERON', 'anatomy_protein_present'): 'BGEE',
#     ('NCBI', 'UBERON', 'anatomy_protein_absent'): 'BGEE',
#     ('NCBI', 'REACTOME', 'pathway_protein'): 'Reactome',
#     #('DrugBank', 'MONDO', 'contraindication'): 'DrugCentral', ## Note: need to double check if other relation types are for this data source
#     #('DrugBank', 'MONDO', 'indication'): 'DrugCentral', ## note: there is a typo? of DrugCentral into DiseaseCentral in their comments
#     ('DrugBank', 'MONDO', 'off-label use'): 'DrugCentral',
#     ('DrugBank', 'NCBI', 'drug_protein'): 'DrugBank',
#     ('DrugBank', 'DrugBank', 'drug_drug'): 'DrugBank',
#     ('DrugBank', 'HPO', 'drug_effect'): 'SIDER',
#     ('MONDO', 'MONDO', 'disease_disease'): 'MONDO',
#     ('MONDO', 'NCBI', 'disease_protein'): 'MONDO',
#     ('MONDO', 'HPO', 'disease_phenotype_positive'): 'HPO-A',
#     ('MONDO', 'HPO', 'disease_phenotype_negative'): 'HPO-A',
#     ('HPO', 'HPO', 'phenotype_phenotype'): 'HPO',
#     ('GO', 'GO', 'bioprocess_bioprocess'): 'GO',
#     ('GO', 'GO', 'molfunc_molfunc'): 'GO',
#     ('GO', 'GO', 'cellcomp_cellcomp'): 'GO',
#     ('CTD', 'NCBI', 'exposure_protein'): 'CTD',
#     ('CTD', 'MONDO', 'exposure_disease'): 'CTD',
#     ('CTD', 'CTD', 'exposure_exposure'): 'CTD',
#     ('CTD', 'GO', 'exposure_bioprocess'): 'CTD',
#     ('CTD', 'GO', 'exposure_molfunc'): 'CTD',
#     ('CTD', 'GO', 'exposure_cellcomp'): 'CTD',
#     ('UBERON', 'UBERON', 'anatomy_anatomy'): 'UBERON',
#     ('REACTOME', 'REACTOME', 'pathway_pathway'): 'Reactome',
# }

## Map the combination to a new column 'knowledge_source'
## if not finding a mapped original data source then label as the edge is from PrimeKG itself
# edges_deep_copy_df['knowledge_source'] = edges_deep_copy_df.apply(
#     lambda row: data_source_map_dict.get((row['x_source'], row['y_source'], row['relation']), 'PrimeKG'), axis=1)

edges_deep_copy_df2['knowledge_source'] = 'PrimeKG'

In [194]:
# Count occurrences of each unique value in 'knowledge_source'
counts = edges_deep_copy_df2['knowledge_source'].value_counts()

print(counts)

knowledge_source
PrimeKG    4009644
Name: count, dtype: int64


In [ ]:
## Sanity check those rows with PrimeKG as datasource
## and randomly select 1 row out of each relation type

# sampled_df = edges_deep_copy_df2[edges_deep_copy_df2['knowledge_source'] == 'PrimeKG'].groupby(['relation', 'x_source', 'y_source']).sample(n=1, random_state=151)  # Set random_state for reproducibility
# sampled_df

In [226]:
## add a new knowledge_souce column and set value to be "PrimeKG"
edges_deep_copy_df2['knowledge_souce_total'] = 'PrimeKG'
## add a new knowledge_level column and set value to be 'knowledge_assertion'
edges_deep_copy_df2['knowledge_level'] = 'knowledge_assertion'
## add a new agent_type column and set value to be 'manual_agent'
edges_deep_copy_df2['agent_type'] = 'automated_agent'

## copy to a final df
edge_df = edges_deep_copy_df2.copy(deep = True)

print(edge_df.shape[0])
## Remove rows where subject or object is empty
# Remove rows where 'Subject' OR 'Object' have NaN values
edge_df = edge_df.dropna(subset=['subject', 'object'])

print(edge_df.shape[0])

edge_df['deploy_date'] = deployment_date
# edge_df.head(5)

4009644
4009644


In [227]:
## rename those columns into desired format
edge_df.rename(columns={'x_name': 'subject_name', 'y_name': 'object_name'}, inplace=True)

## create a publication column and fill na
## if all of them are empty then fill na
edge_df['publications'] = np.nan

In [228]:
## create a context_qualifier column and fill na
## if all of them are empty then fill na
edge_df['context_qualifier'] = np.nan
# edge_df.head(5)

### Get all helper functions

In [229]:
## Load all helper functions
%run /Users/Weiqi0/ISB_working/Hadlock_lab/QI_ISB_Git_repo/TranslatorPharcogenomicsKG/Parser_helper_functions.ipynb

In [230]:
### Add resources_id column, checking whether edge is already
column_list = ['subject', 'predicate', 'object', 'context_qualifier', 'deploy_date']
# Apply the function to each row to generate UUIDs
edge_df['id'] = edge_df[column_list].apply(generate_uuid, axis=1)

# edge_df['id'] = generate_uuid_from_columns(edge_df, column_list)
# edge_df.head(5)

### Now create the corresponding node file
* only need three columns: id, name, category

In [231]:
node_subject_df = edge_df[['subject', 'subject_name', 'subject_category']]
node_object_df = edge_df[['object', 'object_name', 'object_category']]

## rename those columns into desired format
node_subject_df.rename(columns={'subject': 'id', 'subject_name': 'name', 'subject_category': 'category'}, inplace=True)
node_object_df.rename(columns={'object': 'id', 'object_name': 'name', 'object_category': 'category'}, inplace=True)

concat_node_df = pd.concat([node_subject_df, node_object_df]).drop_duplicates(keep='first')

concat_node_df.head()

/var/folders/ml/cjwyk2ps62361rr96_3b_n080000gp/T/ipykernel_17326/1611443582.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  node_subject_df.rename(columns={'subject': 'id', 'subject_name': 'name', 'subject_category': 'category'}, inplace=True)
/var/folders/ml/cjwyk2ps62361rr96_3b_n080000gp/T/ipykernel_17326/1611443582.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  node_object_df.rename(columns={'object': 'id', 'object_name': 'name', 'object_category': 'category'}, inplace=True)


,id,name,category
0,NCBIGene:9796,PHYHIP,biolink:Gene
1,NCBIGene:7918,GPANK1,biolink:Gene
2,NCBIGene:8233,ZRSR2,biolink:Gene
3,NCBIGene:4899,NRF1,biolink:Gene
4,NCBIGene:5297,PI4KA,biolink:Gene


In [232]:
## drop no longer needed columns from edge df
print(edge_df.columns.tolist())
drop_cols = ['relation', 'display_relation', 'x_index', 'x_id', 'x_type',  'x_source', 'y_index', 'y_id', 'y_type', 'y_source']

edge_output_df = edge_df.drop(drop_cols, axis=1)

# edge_output_df.head()

['relation', 'display_relation', 'x_index', 'x_id', 'x_type', 'subject_name', 'x_source', 'y_index', 'y_id', 'y_type', 'object_name', 'y_source', 'subject', 'object', 'subject_category', 'object_category', 'predicate', 'subject_direction_qualifier', 'object_direction_qualifier', 'object_aspect_qualifier', 'knowledge_source', 'knowledge_souce_total', 'knowledge_level', 'agent_type', 'deploy_date', 'publications', 'context_qualifier', 'id']


In [233]:
## exclude rows with Null and empty string values in the predicate column
# edge_output_df = edge_output_df.dropna(subset=['predicate'])

## Drop rows where 'name' is NaN, None, or empty string
edge_output_df = edge_output_df[~edge_output_df['predicate'].isna() & (edge_output_df['predicate'].str.strip() != '')]

## throw away those rows which either subject_category, object_category, or predicate is not start with "biolink:" prefix
## since they cannot be biolink-compliant converted
## Keep only rows where all three columns start with 'biolink:'
# edge_output_df = edge_output_df[
#     edge_output_df['subject_category'].str.startswith('biolink:') &
#     edge_output_df['object_category'].str.startswith('biolink:') &
#     edge_output_df['predicate'].str.startswith('biolink:')
# ]

In [234]:
## Check the knowledge_source column again
## Count occurrences of each unique value in 'knowledge_source'
counts = edge_output_df['knowledge_source'].value_counts()

print(counts)

knowledge_source
PrimeKG    954592
Name: count, dtype: int64


In [235]:
print(len(edge_output_df))

954592


In [236]:
## sanity on the column names of the parsed KG
## the desired order should be: 
## 'subject', 'object', 'subject_category', 'object_category', 'predicate', 'knowledge_source', 'knowledge_souce_total', 
## 'knowledge_level', 'agent_type', 'deploy_date', 'context_qualifier', 'id'

## Print the order of column names
print(list(edge_output_df.columns))

['subject_name', 'object_name', 'subject', 'object', 'subject_category', 'object_category', 'predicate', 'subject_direction_qualifier', 'object_direction_qualifier', 'object_aspect_qualifier', 'knowledge_source', 'knowledge_souce_total', 'knowledge_level', 'agent_type', 'deploy_date', 'publications', 'context_qualifier', 'id']


In [237]:
## Group by predicate
grouped = edge_output_df.groupby('predicate')

## For each predicate, output unique (subject_category, object_category) pairs
for predicate, group in grouped:
    print(f"\nPredicate: {predicate}")
    pairs = group[['subject_category', 'object_category']].drop_duplicates()
    for _, row in pairs.iterrows():
        print(f"  ({row['subject_category']}, {row['object_category']})")


Predicate: biolink:GeneAffectsChemicalAssociation
  (biolink:Gene, biolink:Drug)

Predicate: biolink:broad_match
  (biolink:PhenotypicFeature, biolink:PhenotypicFeature)
  (biolink:Disease, biolink:Disease)

Predicate: biolink:can_be_carried_out_by
  (biolink:Gene, biolink:Drug)

Predicate: biolink:genetically_associated_with
  (biolink:Gene, biolink:PhenotypicFeature)
  (biolink:Gene, biolink:Disease)

Predicate: biolink:has_contraindication
  (biolink:Drug, biolink:Disease)

Predicate: biolink:has_phenotype
  (biolink:Disease, biolink:PhenotypicFeature)

Predicate: biolink:physical_interacts_with
  (biolink:Gene, biolink:Gene)

Predicate: biolink:target_for
  (biolink:Gene, biolink:Drug)

Predicate: biolink:treats
  (biolink:Drug, biolink:Disease)


In [238]:
edge_output_df.head(5)

,subject_name,object_name,subject,object,subject_category,object_category,predicate,subject_direction_qualifier,object_direction_qualifier,object_aspect_qualifier,knowledge_source,knowledge_souce_total,knowledge_level,agent_type,deploy_date,publications,context_qualifier,id
0,PHYHIP,KIF15,NCBIGene:9796,NCBIGene:56992,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,None,None,NaN,PrimeKG,PrimeKG,knowledge_assertion,automated_agent,2026-01-06,NaN,NaN,82779b17-5c64-5e78-b12a-0b0490d1e404
1,GPANK1,PNMA1,NCBIGene:7918,NCBIGene:9240,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,None,None,NaN,PrimeKG,PrimeKG,knowledge_assertion,automated_agent,2026-01-06,NaN,NaN,0210c181-665f-549e-a8e0-6216eb316312
2,ZRSR2,TTC33,NCBIGene:8233,NCBIGene:23548,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,None,None,NaN,PrimeKG,PrimeKG,knowledge_assertion,automated_agent,2026-01-06,NaN,NaN,e8538283-d8d4-5c52-934e-ac94f66cca6e
3,NRF1,MAN1B1,NCBIGene:4899,NCBIGene:11253,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,None,None,NaN,PrimeKG,PrimeKG,knowledge_assertion,automated_agent,2026-01-06,NaN,NaN,dabd6522-e108-5b35-9eba-dcf322c8eddb
4,PI4KA,RGS20,NCBIGene:5297,NCBIGene:8601,biolink:Gene,biolink:Gene,biolink:physical_interacts_with,None,None,NaN,PrimeKG,PrimeKG,knowledge_assertion,automated_agent,2026-01-06,NaN,NaN,90fc7fc7-3a73-5945-a074-8bc47634d23b


In [239]:
## reorder the dataframe
desired_order = [
    'subject', 'subject_name', 'subject_category', 'object', 'object_name', 'object_category', 
    'predicate', 'subject_direction_qualifier', 'object_direction_qualifier', 'object_aspect_qualifier',
    'knowledge_source', 'publications', 'knowledge_level', 'context_qualifier', 'agent_type', 
    'deploy_date', 'id'
]

edge_output_df = edge_output_df[desired_order]

print(edge_output_df.columns.tolist())

['subject', 'subject_name', 'subject_category', 'object', 'object_name', 'object_category', 'predicate', 'subject_direction_qualifier', 'object_direction_qualifier', 'object_aspect_qualifier', 'knowledge_source', 'publications', 'knowledge_level', 'context_qualifier', 'agent_type', 'deploy_date', 'id']


### Now quality control of the parsed PrimeKG dataframe
* Original edge counts: 954592
* QA:
  - dropped duplicated edges in reverse direction
  - dropped enzyme and node types not easily mapped in biolink
  - removed edges with invalid predicate, subject, and object relationship

In [240]:
## Create a graph from the DataFrame
graph = nx.from_pandas_edgelist(edge_output_df, 'subject', 'object', edge_attr='predicate')

## Print graph information
print('Number of nodes', len(set(graph.nodes)))
print('Number of edges', len(set(graph.edges)))
print('Average degree', sum(dict(graph.degree).values()) / len(graph.nodes))

Number of nodes 55770
Number of edges 589950
Average degree 21.15653577192039


In [241]:
## check all unique predicate values
counts = concat_node_df['category'].value_counts()
print(counts)

category
biolink:Gene                 18995
biolink:Disease              15343
biolink:PhenotypicFeature    15271
biolink:Drug                  7944
Name: count, dtype: int64


In [242]:
## check missingness
print(edge_df[['predicate', 'subject_direction_qualifier', 'object_direction_qualifier', 'object_aspect_qualifier']].isna().sum())

predicate                      3055052
subject_direction_qualifier    4009644
object_direction_qualifier     4009644
object_aspect_qualifier        4009644
dtype: int64


In [243]:
## fill na temporily to get the combo number
filled = edge_df[['predicate', 'subject_direction_qualifier', 'object_direction_qualifier', 'object_aspect_qualifier']].fillna('None')
counts = filled.value_counts().reset_index(name='count')
print(counts)

                                predicate subject_direction_qualifier  \
0                                    None                        None   
1         biolink:physical_interacts_with                        None   
2                   biolink:has_phenotype                        None   
3                     biolink:broad_match                        None   
4     biolink:genetically_associated_with                        None   
5            biolink:has_contraindication                        None   
6                      biolink:target_for                        None   
7  biolink:GeneAffectsChemicalAssociation                        None   
8                          biolink:treats                        None   
9           biolink:can_be_carried_out_by                        None   

  object_direction_qualifier object_aspect_qualifier    count  
0                       None                    None  3055052  
1                       None                    None   642150  
2   

## Now download the concatenated node & edge files

In [244]:
## Define the output path for node & edge files after formatting
download_path_node_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/wQualityControl/PrimeKG_parsed_node_{version_number}.tsv'
download_path_edge_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/wQualityControl/PrimeKG_parsed_edge_{version_number}.tsv'

## download both node and edge files
## Download the result df
concat_node_df.to_csv(download_path_node_file, sep ='\t', index=False)
edge_output_df.to_csv(download_path_edge_file, sep ='\t', index=False)

In [245]:
print("The formatted node file will be saved in this path: ", download_path_node_file)
print("The formatted edge file will be saved in this path: ", download_path_edge_file)

The formatted node file will be saved in this path:  /Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/wQualityControl/PrimeKG_parsed_node_01_06_2026.tsv
The formatted edge file will be saved in this path:  /Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/wQualityControl/PrimeKG_parsed_edge_01_06_2026.tsv


In [68]:
# ## throw away those rows which either subject_category, object_category, or predicate is not start with "biolink:" prefix
# ## since they cannot be biolink-compliant converted
# ## Keep only rows where all three columns start with 'biolink:'
# cellmarker_signor_edge_df_filtered = cellmarker_signor_edge_df_filtered[
#     cellmarker_signor_edge_df_filtered['subject_category'].str.startswith('biolink:') &
#     cellmarker_signor_edge_df_filtered['object_category'].str.startswith('biolink:') &
#     cellmarker_signor_edge_df_filtered['predicate'].str.startswith('biolink:')
# ]

# cellmarker_signor_node_df = cellmarker_signor_node_df[
#     cellmarker_signor_node_df['category'].str.startswith('biolink:')]

## Now checking how many duplicates are there in the BigGIM edge file

In [81]:
## Define the version number
BigGIM_version_number = "03_04_2025"
BigGIM_path_edge_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/BigGIM_plover_deployment/files/BigGIM_DrugResponse/biolink_compliant/BigGIM.interacts_with_associated_with.edges.biolink.compliant.tsv'

In [82]:
BigGIM_edge_df = pd.read_csv(BigGIM_path_edge_file, sep = '\t')
BigGIM_edge_df.head()

/var/folders/ml/cjwyk2ps62361rr96_3b_n080000gp/T/ipykernel_71220/4120548939.py:1: DtypeWarning: Columns (5,6,12,13,14,16,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  BigGIM_edge_df = pd.read_csv(BigGIM_path_edge_file, sep = '\t')


,subject,predicate,object,agent_type,knowledge_level,anatomical_context_qualifier,knowledge_source,object_category,publications,subject_category,...,subject_aspect_qualifier,supporting_study_cohort,Data_set,P_value,context_qualifier,statistics_method,supporting_study_size,object_aspect_qualifier,deploy_date,id
0,NCBIGene:10461,biolink:expressed_in,CL:0000235,manual_agent,knowledge_assertion,UBERON_0000916,CellMarker,biolink:Cell,PMID:31982413,biolink:Gene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-01,b041e47f-c4f5-5166-b4fe-557207fcb593
1,NCBIGene:2215,biolink:expressed_in,CL:0000235,manual_agent,knowledge_assertion,UBERON_0000916,CellMarker,biolink:Cell,PMID:31982413,biolink:Gene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-01,99f71540-4efa-55bf-87f1-48cdceb65579
2,NCBIGene:4360,biolink:expressed_in,CL:0000235,manual_agent,knowledge_assertion,UBERON_0000916,CellMarker,biolink:Cell,PMID:31982413,biolink:Gene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-01,9683d6f3-ab01-5331-8a8d-0b1dcd291432
3,NCBIGene:11326,biolink:expressed_in,CL:0000235,manual_agent,knowledge_assertion,UBERON_0000916,CellMarker,biolink:Cell,PMID:31982413,biolink:Gene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-01,b035b572-65d6-5602-ad4e-bbe2c36cf9e9
4,NCBIGene:9332,biolink:expressed_in,CL:0000235,manual_agent,knowledge_assertion,UBERON_0000916,CellMarker,biolink:Cell,PMID:31982413,biolink:Gene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-01,20203999-528e-5335-906e-5441d94ee18c


In [83]:
# Specify the columns to compare
cols = ['subject', 'object', 'predicate']

# Use merge to find common rows based on the specified columns
common = pd.merge(edge_final_df[cols], BigGIM_edge_df[cols], on=cols)

# Get the number of duplicated rows
num_duplicates = len(common)

print(f"Number of duplicated rows: {num_duplicates}")

Number of duplicated rows: 148185


## Now checking how many duplicates are there in the PharmGKB edge file

In [84]:
## Define the version number
PharmGKB_version_number = "07_08_2025"
PharmGKB_path_edge_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/PharmGKB_parsed_edge_{PharmGKB_version_number}.tsv'

PharmGKB_edge_df = pd.read_csv(PharmGKB_path_edge_file, sep = '\t')
PharmGKB_edge_df.head()

,publications,subject,object,subject_category,object_category,predicate,knowledge_source,knowledge_level,agent_type,deploy_date,context_qualifier,id
0,25695618,NCBIGene:162282,MeSH:D000075222,biolink:Gene,biolink:Disease,biolink:associated_with,PharmGKB,knowledge_assertion,automated_agent,2025-07-08,NaN,3f4622cd-cc81-5c60-9fce-4d25214b9b86
1,25695618,NCBIGene:162282,PUBCHEM.COMPOUND:3639,biolink:Gene,biolink:ChemicalEntity,biolink:associated_with,PharmGKB,knowledge_assertion,automated_agent,2025-07-08,NaN,2ebd74fd-1cb6-519c-9051-4e0e5880a57d
2,18511948;25545243,NCBIGene:1728,PUBCHEM.COMPOUND:3385,biolink:Gene,biolink:ChemicalEntity,biolink:associated_with,PharmGKB,knowledge_assertion,automated_agent,2025-07-08,NaN,3e9e9eee-e33e-5903-b6c0-fd6f960ab809
3,24924344;25545243,NCBIGene:1728,PUBCHEM.COMPOUND:6857599,biolink:Gene,biolink:ChemicalEntity,biolink:associated_with,PharmGKB,knowledge_assertion,automated_agent,2025-07-08,NaN,5dfcff5b-dea4-53f7-b9ff-1db8d23309df
4,30237583,NCBIGene:1728,MeSH:D046152,biolink:Gene,biolink:Disease,biolink:associated_with,PharmGKB,knowledge_assertion,automated_agent,2025-07-08,NaN,6149843b-5b1d-562b-b2e9-82391a552db3


In [85]:
## check all unique predicate values
counts = PharmGKB_edge_df['subject_category'].value_counts()
print(counts)

subject_category
biolink:Gene              11702
biolink:ChemicalEntity     5907
biolink:Disease            3349
Name: count, dtype: int64


In [86]:
## check all unique predicate values
counts = PharmGKB_edge_df['subject_category'].value_counts()
print(counts)

subject_category
biolink:Gene              11702
biolink:ChemicalEntity     5907
biolink:Disease            3349
Name: count, dtype: int64


In [87]:
## check all unique predicate values
counts = PharmGKB_edge_df['predicate'].value_counts()

print(counts)

predicate
biolink:associated_with    20958
Name: count, dtype: int64


In [88]:
# Specify the columns to compare
cols = ['subject', 'object', 'predicate']

# Use merge to find common rows based on the specified columns
common = pd.merge(edge_final_df[cols], PharmGKB_edge_df[cols], on=cols)

# Get the number of duplicated rows
num_duplicates = len(common)

print(f"Number of duplicated rows: {num_duplicates}")

Number of duplicated rows: 0


In [89]:
PharmGKB_path_node_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/PharmGKB_parsed_node_{version_number}.tsv'

PharmGKB_node_df = pd.read_csv(PharmGKB_path_node_file, sep = '\t')
PharmGKB_node_df.head()

,id,name,category
0,NCBIGene:162282,ANKFN1,biolink:Gene
1,NCBIGene:1728,NQO1,biolink:Gene
2,NCBIGene:6813,STXBP2,biolink:Gene
3,PUBCHEM.COMPOUND:33,chloroacetaldehyde,biolink:ChemicalEntity
4,NCBIGene:8000,PSCA,biolink:Gene


In [90]:
## Concatenate to obtain overall node & edge dataframes

list_of_input_node_files = [node_final_df, PharmGKB_node_df]

list_of_input_edge_files = [edge_final_df, PharmGKB_edge_df]

node_final_df = pd.concat(list_of_input_node_files).drop_duplicates()
edge_final_df = pd.concat(list_of_input_edge_files).drop_duplicates()

# edge_df.head(10)

In [91]:
## remove NA or empty strings rows from both subject and object columns
# Drop rows where 'name' is NaN, None, or empty string

edge_final_df = edge_final_df[~edge_final_df['subject'].isna() & (edge_final_df['subject'].str.strip() != '')]
edge_final_df = edge_final_df[~edge_final_df['object'].isna() & (edge_final_df['object'].str.strip() != '')]

In [92]:
## Create a graph from the DataFrame
graph = nx.from_pandas_edgelist(edge_final_df, 'subject', 'object', edge_attr='predicate')

## Print graph information
print('Number of nodes', len(set(graph.nodes)))
print('Number of edges', len(set(graph.edges)))
print('Average degree', sum(dict(graph.degree).values()) / len(graph.nodes))

Number of nodes 52602
Number of edges 599515
Average degree 22.794380441808297


In [105]:
## Define the output path for node & edge files after formatting
download_path_node_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/Pharmagenomics_KG_parsed_node_{version_number}.tsv'
download_path_edge_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/Pharmagenomics_KG_parsed_edge_{version_number}.tsv'

## download both node and edge files
## Download the result df
## disable download for testing
node_final_df.to_csv(download_path_node_file, sep ='\t', index=False)
edge_final_df.to_csv(download_path_edge_file, sep ='\t', index=False)

In [94]:
print("The formatted node file will be saved in this path: ", download_path_node_file)
print("The formatted edge file will be saved in this path: ", download_path_edge_file)

The formatted node file will be saved in this path:  /Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/Pharmagenomics_KG_parsed_node_07_08_2025.tsv
The formatted edge file will be saved in this path:  /Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/parsed/Pharmagenomics_KG_parsed_edge_07_08_2025.tsv


In [95]:
edge_final_df.columns

Index(['subject', 'object', 'subject_category', 'object_category', 'predicate',
       'knowledge_source', 'knowledge_level', 'agent_type', 'deploy_date',
       'context_qualifier', 'id', 'predicate_source_name',
       'anatomical_context_qualifier', 'publications'],
      dtype='object')

In [96]:
## Check the knowledge_source column again
## Count occurrences of each unique value in 'knowledge_source'
counts = edge_final_df['knowledge_source'].value_counts()

print(counts)

knowledge_source
PrimeKG          914870
PharmGKB          20427
CellMarker         6599
SIGNOR-250870         1
SIGNOR-58928          1
                  ...  
SIGNOR-249677         1
SIGNOR-277555         1
SIGNOR-256127         1
SIGNOR-261145         1
SIGNOR-261887         1
Name: count, Length: 33227, dtype: int64


In [97]:
edge_final_df['knowledge_source'].isna().sum()

np.int64(0)

In [98]:
edge_final_df['id'].isna().sum()

np.int64(0)

In [99]:
edge_final_df['predicate'].isna().sum()

np.int64(0)

In [100]:
edge_final_df['object'].isna().sum()

np.int64(0)

In [101]:
edge_final_df['subject'].isna().sum()

np.int64(0)

## following codes are used for quality control and sanity check
* check and confirm all subject & object types are correctly formatted

In [102]:
## check all unique predicate values
counts = edge_final_df['subject_category'].value_counts()

print(counts)

subject_category
biolink:Gene                     705625
biolink:Disease                  163798
biolink:Drug                      91961
biolink:ChemicalEntity             8166
biolink:MacromolecularComplex      2632
biolink:ProteinFamily              1341
biolink:SmallMolecule              1085
biolink:Protein                     357
biolink:EnvironmentalProcess        109
biolink:MicroRNA                     43
biolink:Noncoding_RNAProduct          3
Name: count, dtype: int64


In [103]:
## check all unique predicate values
counts = edge_final_df['object_category'].value_counts()

print(counts)

object_category
biolink:Gene                          683140
biolink:DiseaseOrPhenotypicFeature    175784
biolink:Disease                        79946
biolink:Drug                           20336
biolink:Cell                            6599
biolink:ChemicalEntity                  5644
biolink:PhenotypicFeature               1573
biolink:MacromolecularComplex           1218
biolink:SmallMolecule                    426
biolink:ProteinFamily                    229
biolink:Protein                          225
Name: count, dtype: int64


In [104]:
## check all unique predicate values
counts = edge_final_df['predicate'].value_counts()

print(counts)

predicate
biolink:physical_interacts_with           642150
biolink:has_phenotype                     111000
biolink:has_side_effect                    64784
biolink:broad_match                        49452
biolink:regulates                          33096
biolink:has_contraindication               24962
biolink:associated_with                    20427
biolink:target_for                         16380
biolink:expressed_in                        6599
biolink:GeneAffectsChemicalAssociation      3092
biolink:treats                              2186
biolink:can_be_carried_out_by                864
biolink:in_complex_with                      128
Name: count, dtype: int64
